In [1]:
import sys
sys.path.append('/home/maximilian.hofmann/ff_utils/src')

In [2]:
from importlib import reload
from itertools import repeat
from date_utils import partition_date_range
import pandas as pd

In [3]:
import functions as fu
reload(fu)

/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_geopandas/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


<module 'functions' from '/home/maximilian.hofmann/tools/pricing-tool-v2/data/functions.py'>

In [4]:
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count

In [5]:
with open('/home/maximilian.hofmann/tools/brand_science_pricing/data/queries/sql_otter_data.sql', 'r') as f:
    sql_template = f.read()

In [6]:
project_name = 'css-operations'
stag_dataset_name = 'brand_science_stag'
stag_table_name = 'pricing_otter_data_stag'
full_stag_table_name = '.'.join([project_name, stag_dataset_name, stag_table_name])

In [7]:
begin_date = '2023-01-01'
end_date = '2023-02-24'
days_per_batch = 10

In [8]:
date_list = partition_date_range(pd.to_datetime(begin_date), pd.to_datetime(end_date), days_per_batch)
date_list

[['2022-12-26', '2023-01-04'],
 ['2023-01-05', '2023-01-14'],
 ['2023-01-15', '2023-01-24'],
 ['2023-01-25', '2023-02-03'],
 ['2023-02-04', '2023-02-13'],
 ['2023-02-14', '2023-02-20']]

In [9]:
country_code = 'BR'
clustering_fields = ['country_code']

In [10]:
num_processes = min(cpu_count(), len(date_list))
num_processes

6

In [ ]:
res = []
with Pool(num_processes) as p:
    r = p.starmap(fu.process_otter_data_date_range, zip(repeat(sql_template), date_list, repeat(country_code), repeat(full_stag_table_name), repeat(clustering_fields)))
    res.append(r)

Start data cleaning...
Start data cleaning...
Start data cleaning...
Start data cleaning...
Start data cleaning...
Start data cleaning...
Data cleaning finished!
Job f86fa676-49c9-4b1a-ab1c-96c5f3bf5658 is currently in state RUNNING
Data cleaning finished!
Job ea00d6b9-05bc-4010-a641-cca97cbc779e is currently in state RUNNING
Job ea00d6b9-05bc-4010-a641-cca97cbc779e is currently in state RUNNING
Data cleaning finished!
Data cleaning finished!
Data cleaning finished!
Job 01228171-a06c-4fff-9f4c-8dfe17ddde85 is currently in state RUNNING
Job 468072de-dc1e-46e1-864e-8e3d78234f06 is currently in state RUNNING
Job 01228171-a06c-4fff-9f4c-8dfe17ddde85 is currently in state RUNNING
Job 468072de-dc1e-46e1-864e-8e3d78234f06 is currently in state RUNNING
Data cleaning finished!
Job 2a125831-a843-4dac-8991-33a26c28fd36 is currently in state RUNNING
Job 2a125831-a843-4dac-8991-33a26c28fd36 is currently in state RUNNING


In [7]:
prod_dataset_name = 'brand_science'
prod_table_name = 'pricing_otter_data'
full_prod_table_name = '.'.join([project_name, prod_dataset_name, prod_table_name])

In [8]:
import time
time.sleep(60)

KeyboardInterrupt: 

In [9]:
create_tbl = fu.create_tbl(full_stag_table_name, full_prod_table_name)

In [10]:
upsert_tbl = fu.upsert_tbl_otter_data(full_stag_table_name, full_prod_table_name)